# Notebook for make a modeling focus in reduce churn rate in Telco SA

# Libs

In [1]:
import pandas as pd
import numpy as np
import pickle 

import seaborn as sns

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline

from yellowbrick.classifier import ConfusionMatrix
from yellowbrick.classifier import ROCAUC

from lazypredict.Supervised import LazyClassifier


# Load - train and val data

In [2]:
with open(r'C:\Users\erico\Documents\projeto-churn\Telco-Customer-Churn\data\X_train.pkl', 'rb') as f:
    X_train = pickle.load(f)

with open(r'C:\Users\erico\Documents\projeto-churn\Telco-Customer-Churn\data\y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)

In [3]:
with open(r'C:\Users\erico\Documents\projeto-churn\Telco-Customer-Churn\data\X_valid.pkl', 'rb') as f:
    X_valid = pickle.load(f)

with open(r'C:\Users\erico\Documents\projeto-churn\Telco-Customer-Churn\data\y_valid.pkl', 'rb') as f:
    y_valid = pickle.load(f)

# Overview train

In [4]:
X_train.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
customerID,,,,,,,,,,,,,,,,,,,
7274-RTAPZ,Male,No,No,No,1,Yes,No,Fiber optic,No,No,No,No,Yes,Yes,Month-to-month,Yes,Electronic check,90.55,90.55
5862-BRIXZ,Male,No,No,No,46,No,No phone service,DSL,Yes,No,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),60.75,2893.40
5195-KPUNQ,Female,Yes,No,No,53,Yes,No,Fiber optic,Yes,Yes,No,Yes,No,Yes,One year,Yes,Mailed check,96.75,5206.55
2000-DHJUY,Female,Yes,Yes,No,70,Yes,Yes,DSL,Yes,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),92.20,6474.45
5847-MXBEO,Male,No,No,No,22,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,20.30,475.10


In [5]:
y_train.head()

customerID
7274-RTAPZ    Yes
5862-BRIXZ     No
5195-KPUNQ     No
2000-DHJUY     No
5847-MXBEO     No
Name: Churn, dtype: object

# Overview val

In [6]:
X_valid.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
customerID,,,,,,,,,,,,,,,,,,,
0468-YRPXN,Male,No,No,No,29,Yes,No,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Credit card (automatic),98.80,2807.10
9944-AEXBM,Male,No,No,No,32,Yes,Yes,Fiber optic,No,No,No,No,No,Yes,Month-to-month,Yes,Bank transfer (automatic),85.65,2664.30
7786-WBJYI,Female,No,No,No,4,Yes,Yes,Fiber optic,No,Yes,No,No,No,No,Month-to-month,No,Bank transfer (automatic),77.95,305.55
3679-XASPY,Female,No,Yes,Yes,1,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Electronic check,19.45,19.45
9391-YZEJW,Female,No,No,No,62,Yes,Yes,Fiber optic,No,No,No,No,Yes,Yes,One year,Yes,Bank transfer (automatic),94.00,5757.20


In [7]:
y_valid.head()

customerID
0468-YRPXN     No
9944-AEXBM     No
7786-WBJYI    Yes
3679-XASPY     No
9391-YZEJW     No
Name: Churn, dtype: object

# Transform categorical data

In [8]:
le = LabelEncoder()

- X_train encoder

In [9]:
categorical_cols = X_train.select_dtypes(include=['object']).columns

In [10]:
X_train[categorical_cols] = X_train[categorical_cols].apply(lambda col: le.fit_transform(col))

In [11]:
X_train.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
customerID,,,,,,,,,,,,,,,,,,,
7274-RTAPZ,1,0,0,0,1,1,0,1,0,0,0,0,2,2,0,1,2,90.55,90.55
5862-BRIXZ,1,0,0,0,46,0,1,0,2,0,2,2,2,2,2,1,0,60.75,2893.40
5195-KPUNQ,0,1,0,0,53,1,0,1,2,2,0,2,0,2,1,1,3,96.75,5206.55
2000-DHJUY,0,1,1,0,70,1,2,0,2,2,2,2,2,2,2,1,0,92.20,6474.45
5847-MXBEO,1,0,0,0,22,1,0,2,1,1,1,1,1,1,1,0,3,20.30,475.10


- X_valid encoder

In [12]:
categorical_cols = X_valid.select_dtypes(include=['object']).columns

In [13]:
X_valid[categorical_cols] = X_valid[categorical_cols].apply(lambda col: le.fit_transform(col))

In [15]:
X_valid.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
customerID,,,,,,,,,,,,,,,,,,,
0468-YRPXN,1,0,0,0,29,1,0,1,0,0,2,2,2,2,0,1,1,98.80,2807.10
9944-AEXBM,1,0,0,0,32,1,2,1,0,0,0,0,0,2,0,1,0,85.65,2664.30
7786-WBJYI,0,0,0,0,4,1,2,1,0,2,0,0,0,0,0,0,0,77.95,305.55
3679-XASPY,0,0,1,1,1,1,0,2,1,1,1,1,1,1,0,0,2,19.45,19.45
9391-YZEJW,0,0,0,0,62,1,2,1,0,0,0,0,2,2,1,1,0,94.00,5757.20
